# Redes Neuronales Artificiales
## Perceptrón

El preceptrón es una estructura que trata de imitar el funcionamiento de una neurona.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/10/Blausen_0657_MultipolarNeuron.png/1024px-Blausen_0657_MultipolarNeuron.png" alt="Neurona" style="width: 400px;"/>

> Fig. 1: [Imágen Neurona Wikipedia](https://en.wikipedia.org/wiki/Neuron) <br>

Donde el modelo se simplifica a: <br>

<img src="https://upload.wikimedia.org/wikipedia/commons/3/31/Perceptron.svg" alt="Perceptrón" style="width: 400px;"/>

> Fig. 2: [Imágen Perceptrón Wikipedia](https://en.wikipedia.org/wiki/Perceptron) <br>

Considerando $\overline{x}=(x_{1}, x_{2},...,x_{n})$ y $\overline{w}=(w_{1}, w_{2},...,w_{n})$: <br>
$$percept(\overline{x})=f(\overline{x} \cdot \overline{w} + b)=f(\sum(x_{i} \cdot x_{i} )+ b)$$
Dependiendo como se seleccione $f(o)$, un perceptrón es simirar a una regresión lineal ($f(o)=o$) o a una regresión lógistica $f(o)=\frac{1}{1+e^{-o}}$. Obviamente, existen otras funciones de activación que se irán discutiendo a lo largo del curso.

Combinando diversos preceptrones en forma paralela se forma una capa de una red neuronal. En este caso, la capa se conoce como una capa densa, ya que todas las salidas de la capa están conectadas con cada entrada. En el caso de una capa, $W$ es una matriz de dimensiones $cantidad\ de\ características$ X $cantidad\ de\ perceptrones$, $b$ es un vector de dimensionalidad $cantidad\ de\ perceptrones$, y $f(o)$ se aplíca elemento a elemento del resultado. Considerando el trabajo práctico de la clase anterior $W$ es una matriz de 786 X 10, y $b$ es un vector de 10 elementos.

## Keras
[Keras](https://keras.io/) en una librería de alto nivel para redes neuronales que abstrae las operaciones más comunes de las redes neuronales facilitando la escritura de un código más limpio. Keras está construida sobre [TensorFlow](https://keras.io/backend/), por lo que no es necesario instalar ningún elemento extra. Todas las abstracciones de Keras se encuentran en el paquete tensorflow.keras.

A continuación, se muesta un ejemplo de utilización de Keras para el problema clasificación de MNIST.


In [ ]:
%matplotlib inline 
import matplotlib as mpl 
import seaborn as sn 
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd

from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, GaussianNoise, GaussianDropout, LayerNormalization
from tensorflow.keras.models import Model 
from tensorflow.keras.optimizers import SGD 
from tensorflow.keras.datasets import mnist 
from tensorflow.keras.regularizers import L2
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import confusion_matrix, classification_report, mean_squared_error, mean_absolute_error
from sklearn.datasets import make_moons

def show_confusion_matrix(cm, labels): 
    df_cm = pd.DataFrame(cm, index=labels, columns=labels) 
    sn.heatmap(df_cm, annot=True, fmt="d") 
    plt.show()

mpl.rcParams['figure.figsize'] = [12.0, 8.0] 
print(mpl.rcParams['figure.figsize'])

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print('100 primeros elementos del conjunto de entrenaimento')
f = plt.figure(111)
for i in range(10):
    for j in range(10):
        ax = f.add_subplot(10, 10, i + j*10 + 1)
        ax.set_xticklabels('')
        ax.set_yticklabels('')
        ax.imshow(x_train[i + j*10, :, :], cmap='gray')
plt.show()
print(y_train[:100])

size = x_train.shape[1]*x_train.shape[2]
x_train = x_train.reshape((x_train.shape[0], size)) / 255
x_test = x_test.reshape((x_test.shape[0], size)) / 255

yc_train, yc_test = to_categorical(y_train), to_categorical(y_test)

## Keras capas y modelos
Keras nos permite definir nuestro modelo a través de la combinación de diversas capas en un modelo. Para comenzar, definiremos un modelo basado en una regresión simple y una activación softmax (como en la clase pasada). Para esot utilizaremos dos tipos de capas:

* Input: esta capa nos permite definir la entrada de nuestro modelo.
*Dense: es una capa que multiplica las entradas por los pesos y les suma los bias para finalmente aplicar una función de activación.

Además, definimos el modelo. En este caso, le decimos cual es nuestra entrada y salida. Adicionalmente, lo compilamos indicando la función de perdida y que optimizador utilizaremos. Es importante destacar que como utilizamos elementos estándar de Keras podemos definirlo simplemente con un string, pero podemos definir cualquier función o comportamiento que desearamos.


In [ ]:
i = Input((size,))
d = Dense(10, activation='softmax')(i)

model = Model(inputs=i, outputs=d)
model.compile(loss='categorical_crossentropy', optimizer='sgd')
model.summary()

model.fit(x_train, yc_train, batch_size=50, epochs=100)

In [ ]:
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)
show_confusion_matrix(confusion_matrix(y_test, y_pred), list(map(str, range(10))))
print(classification_report(y_test, y_pred))

En este ejemplo podemos ver como cambiamos los parámetros para el algoritmo de optimización.

In [ ]:
i = Input((size,))
d = Dense(10, activation='softmax')(i)

model = Model(inputs=i, outputs=d)
model.compile(loss='categorical_crossentropy', optimizer=SGD(learning_rate=0.01, momentum=0.9))
model.summary()

model.fit(x_train, yc_train, batch_size=50, epochs=100)

In [ ]:
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)
show_confusion_matrix(confusion_matrix(y_test, y_pred), list(map(str, range(10))))
print(classification_report(y_test, y_pred))

# El problema del Xor
Como se describió en las Slides, la función Xor no puede ser aprendida por una regresión lógistica.

| $X_0$ | $X_1$ | $Y$ |
| --- | --- | --- |
| 0 | 0 | 0 |
| 0 | 1 | 1 |
| 1 | 0 | 1 |
| 1 | 1 | 0 |

In [ ]:
x = np.asarray([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.asarray([0, 1, 1, 0])

In [ ]:
plt.scatter(x[:, 0], x[:, 1], c=['green' if i==1 else 'red' for i in y])
plt.show()

In [ ]:
i = Input((2,))
d = Dense(1, activation='sigmoid')(i)
model = Model(i, d)
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])

h = model.fit(x, y, epochs=1000, verbose=0)

plt.title('Función de perdida')
plt.xlabel('Epochs')
plt.ylabel('Binary Crossentropy')
plt.plot(h.history['loss'])
plt.show()

plt.title('Metrica accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.plot(h.history['accuracy'])
plt.show()
print(model.predict(x))

# Perceptrón multicapas
¿Qué pasa si en vez de usar una sola función lineal concatenamos 2 funciones lineales?

La red neuronal más sencilla, conocida como perceptrón multi-capa, no es más que capas de preceptrones aplicadas una sobre la otra.

Entonces un preceptron multicapas tiene la siguiente forma:

$$l_{1}=f_{1}(\overline{x} \cdot W_{1} + \overline{bias_{1}})$$

$$l_{2}=f_{2}(\overline{l_{1}} \cdot W_{2} + \overline{bias_{2}})$$

$$...$$

$$l_{N}=f_{N}(\overline{l_{N-1}} \cdot W_{N} + \overline{bias_{N}})$$

Es importante destacar que, dado una función de error, calcular el gradiente para cada parámetro de la red, sea $W_{i}$ o $bias_{i}$, es simplemente aplicar la regla de la cadena en repetidas oscaciones. Esto hace que puedan ser calculados de forma automática por librerias como Tensorflow.

In [ ]:
i = Input((2,))
d = Dense(30, activation='tanh')(i)
d = Dense(1, activation='sigmoid')(d)
model = Model(i, d)
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])

h = model.fit(x, y, epochs=500, verbose=0)

plt.title('Función de perdida')
plt.xlabel('Epochs')
plt.ylabel('Binary Crossentropy')
plt.plot(h.history['loss'])
plt.show()

plt.title('Metrica accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.plot(h.history['accuracy'])
plt.show()
print(model.predict(x))

## Overfitting
Uno de los grandes problemas de las redes neuronales, en especial cuando hay pocos datos de entrenamiento, es que tienden a sobreaprender los datos en el conjunto de entrenamiento y no generalizan bien.

Supongamos el caso artificial de la clase anterior donde los datos tiene la forma:

$$y = 3*x + rand(-0.5, 0.5)$$

Obviamente un modelo lineal sería más que suficiente para aprender este conjunto de datos.


In [ ]:
np.random.seed(42)
'''def gen_random_data(mult):
    _x = np.linspace(-1, 1, 20)
    _error = 0.5 * (np.random.rand(*_x.shape) - .5)
    _y = _x * mult + _error
    return _x, _y


x, y = gen_random_data(3)
plt.plot(x, y, 'ro')
plt.show()'''
def gen_random_data(mult):
    _x = np.linspace(-1, 1, 100)
    _error = (np.random.rand(*_x.shape) - .5)
    _y = _x * mult + _error
    return _x, _y


x, y = gen_random_data(3)
plt.plot(x, y, 'ro')
plt.show()
print('x: {}'.format(x))
print('y: {}'.format(y))

In [ ]:
i = Input((1,))
d = Dense(1)(i)
model = Model(i, d)
model.compile(loss='mse', optimizer='sgd')

h = model.fit(np.expand_dims(x, axis=-1), y, epochs=300, verbose=0)
plt.plot(x, y, 'ro', x, model.predict(np.expand_dims(x, axis=-1)))
plt.show()

Pero, si aplicamos un modelo con más capas, este modelo tiende a aprender los "errores" en las observacciones del conjunto de entrenamiento. Con lo que surge la pregunta ¿Cuál es el mejor modelo el representado por la linea azul o la linea verde?

In [ ]:
i = Input((1,))
d = Dense(2, activation='sigmoid')(i)
d = Dense(1)(d)
model1 = Model(i, d)
model1.compile(loss='mse', optimizer='sgd')

h = model1.fit(np.expand_dims(x, axis=-1), y, epochs=1000, verbose=0, callbacks=[ReduceLROnPlateau('loss')])
plt.plot(x, y, 'ro', x, model1.predict(np.expand_dims(x, axis=-1)),'g-' ,x, model.predict(np.expand_dims(x, axis=-1)), 'b-')
plt.show()

## Regularizaciones

En las siguientes celdas se muestran ejemplos de técnicas de regularización generalmente utilizadas en las redes neuronales. Es importante destacar que no son las únicas técnicas que se utilizan para evitar el overfitting.

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1]*X_train.shape[2]) 
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1]*X_test.shape[2])
X_train = X_train/255
X_test = X_test/255

Y_train = to_categorical(y_train) 
Y_test = to_categorical(y_test)

In [ ]:
i = Input((X_train.shape[1],))
d = Dense(512, activation='relu')(i)
d = Dense(256, activation='relu')(d)
d = Dense(128, activation='relu')(d)
d = Dense(10, activation='softmax')(d)
model = Model(i, d)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(X_train, Y_train, epochs=20, verbose=1, validation_data=(X_test, Y_test))

In [ ]:
plt.plot(range(1, 21), history.history['loss'], 'r-',label='Training')
plt.plot(range(1, 21), history.history['val_loss'], 'b-', label='Test')
plt.legend(framealpha=1, frameon=True)
plt.xlabel('Epoch')
plt.ylabel('Loss: Categorical CrossEntropy')
plt.show()

In [ ]:
plt.plot(range(1, 21), history.history['accuracy'], 'r-',label='Training')
plt.plot(range(1, 21), history.history['val_accuracy'], 'b-', label='Test')
plt.legend(framealpha=1, frameon=True)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
i = Input((X_train.shape[1],))
d = Dense(512, activation='relu', kernel_regularizer=L2(0.0001))(i)
d = Dense(256, activation='relu', kernel_regularizer=L2(0.0001))(d)
d = Dense(128, activation='relu', kernel_regularizer=L2(0.0001))(d)
d = Dense(10, activation='softmax', kernel_regularizer=L2(0.0001))(d)
model = Model(i, d)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
history = model.fit(X_train, Y_train, epochs=20, verbose=1, validation_data=(X_test, Y_test))

In [ ]:
plt.plot(range(1, 21), history.history['loss'], 'r-',label='Training')
plt.plot(range(1, 21), history.history['val_loss'], 'b-', label='Test')
plt.legend(framealpha=1, frameon=True)
plt.xlabel('Epoch')
plt.ylabel('Loss: Categorical CrossEntropy')
plt.show()

In [ ]:
plt.plot(range(1, 21), history.history['accuracy'], 'r-',label='Training')
plt.plot(range(1, 21), history.history['val_accuracy'], 'b-', label='Test')
plt.legend(framealpha=1, frameon=True)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
i = Input((X_train.shape[1],))
d = Dense(512, activation='relu')(i)
d = BatchNormalization()(d)
d = Dropout(0.5)(d)
d = Dense(256, activation='relu')(d)
d = BatchNormalization()(d)
d = Dropout(0.5)(d)
d = Dense(128, activation='relu')(d)
d = BatchNormalization()(d)
d = Dropout(0.5)(d)
d = Dense(10, activation='softmax')(d)
model = Model(i, d)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
history = model.fit(X_train, Y_train, epochs=20, verbose=1, validation_data=(X_test, Y_test))

In [ ]:
plt.plot(range(1, 21), history.history['loss'], 'r-',label='Training')
plt.plot(range(1, 21), history.history['val_loss'], 'b-', label='Test')
plt.legend(framealpha=1, frameon=True)
plt.xlabel('Epoch')
plt.ylabel('Loss: Categorical CrossEntropy')
plt.show()

In [ ]:
plt.plot(range(1, 21), history.history['accuracy'], 'r-',label='Training')
plt.plot(range(1, 21), history.history['val_accuracy'], 'b-', label='Test')
plt.legend(framealpha=1, frameon=True)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

## ¿Cuál es el efecto de las diferentes capas de regularización?

En las siguientes celdas se muestra el efecto de la regularización.

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train/255
X_test = X_test/255

d = Dropout(0.5)
X_reg = d(X_train.astype(np.float32), training=True)


f = plt.figure(111)
for i in range(10):
    for j in range(10):
        ax = f.add_subplot(10, 10, i + j*10 + 1)
        ax.set_xticklabels('')
        ax.set_yticklabels('')
        ax.imshow(X_train[i + j*10, :, :], cmap='gray')
plt.show()
print(y_train[:100])

f = plt.figure(111)
for i in range(10):
    for j in range(10):
        ax = f.add_subplot(10, 10, i + j*10 + 1)
        ax.set_xticklabels('')
        ax.set_yticklabels('')
        ax.imshow(X_reg[i + j*10, :, :], cmap='gray')
plt.show()
print(y_train[:100])

In [ ]:
d = GaussianNoise(0.3)
X_reg = d(X_train.astype(np.float32), training=True)


f = plt.figure(111)
for i in range(10):
    for j in range(10):
        ax = f.add_subplot(10, 10, i + j*10 + 1)
        ax.set_xticklabels('')
        ax.set_yticklabels('')
        ax.imshow(X_reg[i + j*10, :, :], cmap='gray')
plt.show()
print(y_train[:100])

In [ ]:
d = GaussianDropout(0.5)
X_reg = d(X_train.astype(np.float32), training=True)


f = plt.figure(111)
for i in range(10):
    for j in range(10):
        ax = f.add_subplot(10, 10, i + j*10 + 1)
        ax.set_xticklabels('')
        ax.set_yticklabels('')
        ax.imshow(X_reg[i + j*10, :, :], cmap='gray')
plt.show()
print(y_train[:100])

In [ ]:
d = BatchNormalization()
X_reg = d(X_train.astype(np.float32), training=True)


f = plt.figure(111)
for i in range(10):
    for j in range(10):
        ax = f.add_subplot(10, 10, i + j*10 + 1)
        ax.set_xticklabels('')
        ax.set_yticklabels('')
        ax.imshow(X_reg[i + j*10, :, :], cmap='gray')
plt.show()
print(y_train[:100])

In [ ]:
d = LayerNormalization()
X_reg = d(X_train.astype(np.float32), training=True)


f = plt.figure(111)
for i in range(10):
    for j in range(10):
        ax = f.add_subplot(10, 10, i + j*10 + 1)
        ax.set_xticklabels('')
        ax.set_yticklabels('')
        ax.imshow(X_reg[i + j*10, :, :], cmap='gray')
plt.show()
print(y_train[:100])

## Early Stopping

Otra técnica bastante usada para disminuir el efecto de overfitting es llamado Early Stopping. Básicamente consiste en dejar de entrenar cuando se comienza a haber overfitting.

In [ ]:
X, Y = make_moons(100, True, 0.2, random_state=42)

for x, y in zip(X, Y):
    if y == 0:
        plt.plot(x[0], x[1], 'r*')
    else:
        plt.plot(x[0], x[1], 'b*')

plt.show()

In [ ]:
n_train = 30
x_train, x_test = X[:n_train, :], X[n_train:, :]
y_train, y_test = Y[:n_train], Y[n_train:]

for x, y in zip(x_train, y_train):
    if y == 0:
        plt.plot(x[0], x[1], 'r*')
    else:
        plt.plot(x[0], x[1], 'b*')

plt.show()


for x, y in zip(x_test, y_test):
    if y == 0:
        plt.plot(x[0], x[1], 'r*')
    else:
        plt.plot(x[0], x[1], 'b*')

plt.show()

In [ ]:
i = Input((2,))
d = Dense(500, activation='relu')(i)
d = Dense(1, activation='sigmoid')(d)

model = Model(i, d)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
h = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=2000, verbose=0)

In [ ]:
_, train_acc = model.evaluate(x_train, y_train, verbose=0)
_, test_acc = model.evaluate(x_test, y_test, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

In [ ]:
plt.plot(h.history['loss'], 'r-', label='train')
plt.plot(h.history['val_loss'], 'b-', label='test')
plt.ylabel('Crossentropy')
plt.xlabel('Epochs')
plt.legend(framealpha=1, frameon=True)
plt.show()

In [ ]:
plt.plot(history.history['accuracy'], 'r-',label='Training')
plt.plot(history.history['val_accuracy'], 'b-', label='Test')
plt.legend(framealpha=1, frameon=True)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
i = Input((2,))
d = Dense(500, activation='relu')(i)
d = Dense(1, activation='sigmoid')(d)

model = Model(i, d)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
h = model.fit(x_train, y_train, validation_data=(x_test, y_test), 
              epochs=2000, verbose=0, callbacks=[EarlyStopping('val_loss', mode='min', patience=1)])

plt.plot(h.history['loss'], 'r-', label='train')
plt.plot(h.history['val_loss'], 'b-', label='test')
plt.ylabel('Crossentropy')
plt.xlabel('Epochs')
plt.legend(framealpha=1, frameon=True)
plt.show()
plt.plot(h.history['accuracy'], 'r-',label='Training')
plt.plot(h.history['val_accuracy'], 'b-', label='Test')
plt.legend(framealpha=1, frameon=True)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()